# Continuous Pendulum Control
Zero reward is the best condition for the pendulum control

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from mvp.env_pendulum import PendulumEnv

from itertools import count
import torch
import gym
from gym.envs.registration import register
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
plt.ion()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from gym.wrappers import NormalizeObservation, TransformObservation, NormalizeReward, TransformReward

# Pendulum Control Gym

## Description
The inverted pendulum swingup problem is a classic problem in control theory. The system consists of a pendulum attached at one end to a fixed point, with the other end being free. The pendulum starts in a random position, and the goal is to apply torque to the free end to swing it into an upright position, where its center of gravity is right above the fixed point.

### Pendulum Coordinate System

- **x-y**: Cartesian coordinates of the pendulum’s end in meters.
- **theta**: Angle in radians.
- **tau**: Torque in N·m, defined as positive counter-clockwise.

### Action Space
The action is an ndarray with shape `(1,)` representing the torque applied to the free end of the pendulum.

| Num | Action | Min | Max |
|-----|--------|-----|-----|
| 0   | Torque | -2.0| 2.0 |

### Observation Space
The observation is an ndarray with shape `(3,)` representing the x-y coordinates of the pendulum’s free end and its angular velocity.

| Num | Observation      | Min | Max |
|-----|------------------|-----|-----|
| 0   | x = cos(theta)   | -1.0| 1.0 |
| 1   | y = sin(theta)   | -1.0| 1.0 |
| 2   | Angular Velocity | -8.0| 8.0 |

### Rewards
The reward function is defined as:

$r = -(theta^2 + 0.1 * theta_dt^2 + 0.001 * torque^2)$

where `theta` is the pendulum’s angle normalized between `[-pi, pi]` (with 0 being in the upright position). The minimum reward that can be obtained is `-(pi^2 + 0.1 * 8^2 + 0.001 * 2^2) = -16.2736044`, while the maximum reward is zero (pendulum is upright with zero velocity and no torque applied).

In [4]:
register(
    id='Pendulum-v4',
    entry_point='mvp.env_pendulum:PendulumEnv',
    max_episode_steps=1000)
env = gym.make('Pendulum-v4')

In [ ]:
env.action_space

In [ ]:
env.reward_range

In [ ]:
env.observation_space

# Evaluation of Model

- KL Divergence on Policy (probabilistic distribution of actions outputted by our model) is very important
- Seems to be stack, may need good experience to do good control
- Swinging up is harder and takes longer for pure PPO
- Normalization causes weird behavior, need to figure out why
- When stuck at bottom, seems to be very hard to accomplish

## PPO

In [ ]:
from mvp.ppo_continuous import ActorCritic

path = os.path.join(os.getcwd(), "..", "mvp", "params", "ppo_vector.pth")

env = PendulumEnv(render_mode="human")
n_actions = env.action_space.shape[0]
state, info = env.reset()
n_observations = len(state)
num_eval_episodes = 10

# model = Agent(n_observations, n_actions).to(device)
model = ActorCritic(env)
model.load_state_dict(torch.load(path, map_location=device))
model.eval()

for i_episode in range(num_eval_episodes):
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        env.render()
        # Get action from the model (assuming it outputs action_mean, action_std, and value)
        with torch.no_grad():
            action_mean, _, _ = model(state)
        
        # Take the mean action (no sampling here for deterministic behavior)
        action = action_mean.cpu().numpy()[0]
        print(action)

        observation, reward, terminated, truncated, _ = env.step(action)
        if terminated or truncated:
            print(f"Episode finished after {t+1} timesteps")
            break

        state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
        
env.close()

## FMPPO

Some how just constantly swings around, but does show fast convergence

In [ ]:
from mvp.fmppo_continuous import ActorCriticUPN

path = os.path.join(os.getcwd(), "..", "mvp", "params", "pendulum_fmppo.pth")

env = PendulumEnv(render_mode="human")
n_actions = env.action_space.shape[0]
state, _ = env.reset()
n_observations = len(state)
num_eval_episodes = 10

# Create and load the model
model = ActorCriticUPN(n_observations, n_actions).to(device)
model.load_state_dict(torch.load(path, map_location=device))
model.eval()

for i_episode in range(num_eval_episodes):
    state, _ = env.reset()
    state = torch.FloatTensor(state).unsqueeze(0).to(device)
    for t in count():
        env.render()
        # Get action from the model
        with torch.no_grad():
            action_mean, _, _ = model(state)
        
        # Take the mean action (no sampling here for deterministic behavior)
        action = action_mean.cpu().numpy()[0]
        print(action)

        next_state, reward, terminated, truncated, _ = env.step(action)
        if terminated or truncated:
            print(f"Episode finished after {t+1} timesteps")
            break

        state = torch.FloatTensor(next_state).unsqueeze(0).to(device)
        
env.close()

## DQN

In [ ]:
from mvp.dqn_networks import DQN

path = os.path.join(os.getcwd(), "..", "mvp", "params", "pendulum_dqn_discrete_retrain.pth")

env = PendulumEnv(render_mode="human")
ACTION_MAP = np.linspace(-2, 2, 5)  # 5 actions ranging from -2 to 2
n_actions = len(ACTION_MAP)
state, info = env.reset()
n_observations = len(state)

model = DQN(n_observations, n_actions).to(device)
model.load_state_dict(torch.load(path, map_location=device))
model.eval()

num_eval_episodes = 10
for i_episode in range(num_eval_episodes):
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        env.render()

        action_idx = model(state).max(1)[1]
        actual_action = ACTION_MAP[action_idx.item()]
        print(action)

        observation, reward, terminated, truncated, _ = env.step([actual_action])
        if terminated or truncated:
            print(f"Episode finished after {t+1} timesteps")
            break

        state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
        
env.close()

## Reward Post-training

In [ ]:
from mvp.ppo_continuous import ActorCritic  # PPO Model
from mvp.fmppo_continuous import ActorCriticUPN  # FM-PPO Model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters for evaluation
num_eval_episodes = 500  # Number of episodes for evaluation
max_timesteps = 200  # Max timesteps per episode

# Function to evaluate a model and collect rewards
def evaluate_model(model, env, num_eval_episodes, max_timesteps):
    all_rewards = []
    model.eval()  # Set the model to evaluation mode

    for i_episode in range(num_eval_episodes):
        state, _ = env.reset()
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        episode_rewards = 0

        for t in range(max_timesteps):
            with torch.no_grad():
                action_mean, _, _ = model(state)

            # Take the mean action (no sampling for deterministic behavior)
            action = action_mean.cpu().numpy()[0]

            # Step the environment
            next_state, reward, terminated, truncated, _ = env.step(action)
            episode_rewards += reward

            if terminated or truncated:
                break

            state = torch.FloatTensor(next_state).unsqueeze(0).to(device)

        all_rewards.append(episode_rewards)

    return all_rewards

# Load the PPO model
ppo_model_path = os.path.join(os.getcwd(), "..", "mvp", "params", "ppo_ITER_3000_KL_0.01_RUN_1.pth")
ppo_model = ActorCritic(n_states=len(env.reset()[0]), n_actions=env.action_space.shape[0]).to(device)
ppo_model.load_state_dict(torch.load(ppo_model_path, map_location=device))

# Load the FM-PPO model (with UPN)
fmppo_model_path = os.path.join(os.getcwd(), "..", "mvp", "params", "pendulum_fmppo.pth")
fmppo_model = ActorCriticUPN(n_states=len(env.reset()[0]), n_actions=env.action_space.shape[0]).to(device)
fmppo_model.load_state_dict(torch.load(fmppo_model_path, map_location=device))

# Set up the environment (without rendering)
env = PendulumEnv(render_mode=None)

# Evaluate PPO Model
ppo_rewards = evaluate_model(ppo_model, env, num_eval_episodes, max_timesteps)

# Evaluate FM-PPO Model (UPN-based)
fmppo_rewards = evaluate_model(fmppo_model, env, num_eval_episodes, max_timesteps)

# Calculate average rewards for both models
average_ppo_reward = sum(ppo_rewards) / num_eval_episodes
average_fmppo_reward = sum(fmppo_rewards) / num_eval_episodes

# Plot the rewards for both models
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_eval_episodes + 1), ppo_rewards, label="PPO Episode Reward")
plt.plot(range(1, num_eval_episodes + 1), fmppo_rewards, label="FM-PPO Episode Reward")
plt.axhline(y=average_ppo_reward, color='r', linestyle='--', label=f'PPO Avg Reward = {average_ppo_reward:.2f}')
plt.axhline(y=average_fmppo_reward, color='g', linestyle='--', label=f'FM-PPO Avg Reward = {average_fmppo_reward:.2f}')
plt.title("PPO vs FM-PPO Episode Rewards over Multiple Evaluations")
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Hyperparameters for evaluation
num_eval_episodes = 500  # Number of episodes for evaluation
max_timesteps = 200  # Max timesteps per episode
eval_interval = 50  # Number of episodes to average over

# Function to evaluate a model and collect rewards
def evaluate_model(model, env, num_eval_episodes, max_timesteps, eval_interval):
    all_rewards = []
    model.eval()  # Set the model to evaluation mode

    for i_episode in range(num_eval_episodes):
        state, _ = env.reset()
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        episode_rewards = 0

        for t in range(max_timesteps):
            with torch.no_grad():
                action_mean, _, _ = model(state)

            # Take the mean action (no sampling for deterministic behavior)
            action = action_mean.cpu().numpy()[0]

            # Step the environment
            next_state, reward, terminated, truncated, _ = env.step(action)
            episode_rewards += reward

            if terminated or truncated:
                break

            state = torch.FloatTensor(next_state).unsqueeze(0).to(device)

        all_rewards.append(episode_rewards)

    # Calculate average rewards for every 'eval_interval' episodes
    avg_rewards = [sum(all_rewards[i:i+eval_interval]) / eval_interval for i in range(0, num_eval_episodes, eval_interval)]

    return avg_rewards

# Set up the environment (without rendering)
env = PendulumEnv(render_mode=None)

# Load the PPO model
ppo_model_path = os.path.join(os.getcwd(), "..", "mvp", "params", "ppo_ITER_300_KL_0.01_RUN_1.pth")
ppo_model = ActorCritic(n_states=len(env.reset()[0]), n_actions=env.action_space.shape[0]).to(device)
ppo_model.load_state_dict(torch.load(ppo_model_path, map_location=device))

# Load the FM-PPO model (with UPN)
fmppo_model_path = os.path.join(os.getcwd(), "..", "mvp", "params", "pendulum_fmppo.pth")
fmppo_model = ActorCriticUPN(n_states=len(env.reset()[0]), n_actions=env.action_space.shape[0]).to(device)
fmppo_model.load_state_dict(torch.load(fmppo_model_path, map_location=device))

# Evaluate PPO Model
ppo_avg_rewards = evaluate_model(ppo_model, env, num_eval_episodes, max_timesteps, eval_interval)

# Evaluate FM-PPO Model (UPN-based)
fmppo_avg_rewards = evaluate_model(fmppo_model, env, num_eval_episodes, max_timesteps, eval_interval)

# Generate x-axis labels representing each 50-episode interval
x_labels = range(1, num_eval_episodes // eval_interval + 1)

# Plot the average rewards for both models
plt.figure(figsize=(10, 6))
plt.plot(x_labels, ppo_avg_rewards, label="PPO Avg Reward per 50 Episodes", marker='o')
plt.plot(x_labels, fmppo_avg_rewards, label="FM-PPO Avg Reward per 50 Episodes", marker='o')
plt.title("PPO vs FM-PPO Average Rewards over 50 Episode Intervals")
plt.xlabel("Episode Interval (50 episodes each)")
plt.ylabel("Average Total Reward")
plt.legend()
plt.grid()
plt.show()